In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob
from textblob.sentiments import PatternAnalyzer

In [2]:
BH_tweets_during_war = pd.read_csv('dane/BH_tweets_during_war.csv', lineterminator='\n')

In [3]:
BH_tweets_during_war.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video,near,geo,source,user_rt_id,user_rt,retweet_id,retweet_date,translate,trans_src,trans_dest
count,3.926000e+04,3.926000e+04,39260.0,3.926000e+04,39260.000000,39260.000000,39260.000000,39260.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.475369e+18,1.473668e+18,200.0,4.734472e+17,0.621345,1.263398,6.696358,0.336780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2.239573e+16,3.469416e+16,0.0,5.934000e+17,10.372276,41.873463,183.888519,0.472615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.432859e+18,1.875374e+17,200.0,5.440000e+02,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.456450e+18,1.456276e+18,200.0,8.284705e+07,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.478437e+18,1.478401e+18,200.0,2.332024e+09,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.495534e+18,1.495466e+18,200.0,1.121801e+18,0.000000,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.507983e+18,1.507976e+18,200.0,1.507420e+18,1093.000000,6630.000000,32155.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
BH_tweets_during_war.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39260 entries, 0 to 39259
Data columns (total 36 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               39260 non-null  int64  
 1   conversation_id  39260 non-null  int64  
 2   created_at       39260 non-null  object 
 3   date             39260 non-null  object 
 4   time             39260 non-null  object 
 5   timezone         39260 non-null  int64  
 6   user_id          39260 non-null  int64  
 7   username         39260 non-null  object 
 8   name             39260 non-null  object 
 9   place            574 non-null    object 
 10  tweet            39260 non-null  object 
 11  language         39260 non-null  object 
 12  mentions         39260 non-null  object 
 13  urls             39260 non-null  object 
 14  photos           39260 non-null  object 
 15  replies_count    39260 non-null  int64  
 16  retweets_count   39260 non-null  int64  
 17  likes_count 

In [5]:
BH_tweets_during_war.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1507983276836859906,1507837885025816588,2022-03-27 09:30:04 CEST,2022-03-27,09:30:04,200,1330089339322327045,wylecoyote73,Wyle.Coyote,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'BTCGandalf', 'name': '\u20bf...",NaN,NaN,NaN,NaN
1,1507975674736353284,1507975674736353284,2022-03-27 08:59:52 CEST,2022-03-27,08:59:52,200,386652145,lieramarco,Marco Liera,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
2,1507971246486663170,1507922750588915712,2022-03-27 08:42:16 CEST,2022-03-27,08:42:16,200,1401558921135919106,citig2,CitiG,NaN,...,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'bkaellner', 'name': 'Brad Ka...",NaN,NaN,NaN,NaN
3,1507962206356447232,1507962206356447232,2022-03-27 08:06:21 CEST,2022-03-27,08:06:21,200,1503531207279194113,ericbla12839863,"ELROCKOERIC, IS Eric Blackburn",NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1507958944802787330,1507958944802787330,2022-03-27 07:53:23 CEST,2022-03-27,07:53:23,200,1259213974131937282,insiderpurchase,SEC - Insider Purchases,NaN,...,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN


In [6]:
#Add date format
BH_tweets_during_war['date'] = pd.to_datetime(BH_tweets_during_war['date'], format = '%Y-%m-%d')

In [7]:
#Remove tweets writeen in other language than english
BH_tweets_during_war_clean = BH_tweets_during_war[BH_tweets_during_war.language == 'en']
BH_tweets_during_war_clean = BH_tweets_during_war_clean.reset_index(drop=True)
BH_tweets_during_war_clean.shape

(35478, 36)

In [8]:
#Remove columns without any data
BH_tweets_during_war_clean = BH_tweets_during_war_clean.dropna(thresh = 1, axis = 1)

In [10]:
#3782 records and 10 columns were removed
BH_tweets_during_war_clean.describe()

,id,conversation_id,timezone,user_id,replies_count,retweets_count,likes_count,video
count,3.547800e+04,3.547800e+04,35478.0,3.547800e+04,35478.000000,35478.000000,35478.000000,35478.000000
mean,1.475095e+18,1.473305e+18,200.0,4.630565e+17,0.617622,1.256863,6.417977,0.346581
std,2.241732e+16,3.536616e+16,0.0,5.899385e+17,10.866320,43.965673,192.735558,0.475888
min,1.432859e+18,1.875374e+17,200.0,5.440000e+02,0.000000,0.000000,0.000000,0.000000
25%,1.456060e+18,1.455862e+18,200.0,7.287155e+07,0.000000,0.000000,0.000000,0.000000
50%,1.478011e+18,1.477764e+18,200.0,2.322035e+09,0.000000,0.000000,0.000000,0.000000
75%,1.495294e+18,1.495162e+18,200.0,1.101810e+18,0.000000,0.000000,1.000000,1.000000
max,1.507983e+18,1.507962e+18,200.0,1.507420e+18,1093.000000,6630.000000,32155.000000,1.000000


In [11]:
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35478 entries, 0 to 35477
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               35478 non-null  int64         
 1   conversation_id  35478 non-null  int64         
 2   created_at       35478 non-null  object        
 3   date             35478 non-null  datetime64[ns]
 4   time             35478 non-null  object        
 5   timezone         35478 non-null  int64         
 6   user_id          35478 non-null  int64         
 7   username         35478 non-null  object        
 8   name             35478 non-null  object        
 9   place            572 non-null    object        
 10  tweet            35478 non-null  object        
 11  language         35478 non-null  object        
 12  mentions         35478 non-null  object        
 13  urls             35478 non-null  object        
 14  photos           35478 non-null  objec

In [12]:
BH_tweets_during_war_clean.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to
0,1507983276836859906,1507837885025816588,2022-03-27 09:30:04 CEST,2022-03-27,09:30:04,200,1330089339322327045,wylecoyote73,Wyle.Coyote,NaN,...,0,0,[],[],https://twitter.com/WyleCoyote73/status/150798...,False,NaN,0,NaN,"[{'screen_name': 'BTCGandalf', 'name': '\u20bf..."
1,1507971246486663170,1507922750588915712,2022-03-27 08:42:16 CEST,2022-03-27,08:42:16,200,1401558921135919106,citig2,CitiG,NaN,...,0,0,[],[],https://twitter.com/CitiG2/status/150797124648...,False,NaN,0,NaN,"[{'screen_name': 'bkaellner', 'name': 'Brad Ka..."
2,1507962206356447232,1507962206356447232,2022-03-27 08:06:21 CEST,2022-03-27,08:06:21,200,1503531207279194113,ericbla12839863,"ELROCKOERIC, IS Eric Blackburn",NaN,...,0,0,[],[],https://twitter.com/EricBla12839863/status/150...,False,NaN,0,NaN,[]
3,1507958944802787330,1507958944802787330,2022-03-27 07:53:23 CEST,2022-03-27,07:53:23,200,1259213974131937282,insiderpurchase,SEC - Insider Purchases,NaN,...,0,0,['occidentalpetroleum'],['oxy'],https://twitter.com/InsiderPurchase/status/150...,False,NaN,0,NaN,[]
4,1507958941203980292,1507958941203980292,2022-03-27 07:53:22 CEST,2022-03-27,07:53:22,200,1259213974131937282,insiderpurchase,SEC - Insider Purchases,NaN,...,0,0,['occidentalpetroleum'],['oxy'],https://twitter.com/InsiderPurchase/status/150...,False,NaN,0,NaN,[]


In [13]:
#Check values of column place
BH_tweets_during_war_clean['place'].value_counts()

{'type': 'Point', 'coordinates': [28.16028, -82.38634]}          39
{'type': 'Point', 'coordinates': [44.6981658, -93.2177749]}      38
{'type': 'Point', 'coordinates': [33.7864851, -84.38743]}        30
{'type': 'Point', 'coordinates': [33.82291021, -116.5135278]}    19
{'type': 'Point', 'coordinates': [44.91360116, -93.32983194]}    16
                                                                 ..
{'type': 'Point', 'coordinates': [42.1653883, -85.5517602]}       1
{'type': 'Point', 'coordinates': [41.89778, -71.95916]}           1
{'type': 'Point', 'coordinates': [38.47389, -121.41686]}          1
{'type': 'Point', 'coordinates': [33.94243, -118.18094]}          1
{'type': 'Point', 'coordinates': [39.967572, -75.1784794]}        1
Name: place, Length: 129, dtype: int64

In [14]:
#Check values of column quote_url
BH_tweets_during_war_clean['quote_url'].value_counts()

https://twitter.com/RBReich/status/1435348653284237318            15
https://twitter.com/YahooFinance/status/1493996148624961542        6
https://twitter.com/bitpanda/status/1485923043209879552            5
https://twitter.com/Dinesh_Sairam/status/1447239466255593483       4
https://twitter.com/DividendGrowth/status/1488150074861490190      4
                                                                  ..
https://twitter.com/CBSLA/status/1468273132314189830               1
https://twitter.com/TrainsMagazine/status/1497958580535709700      1
https://twitter.com/RAHEBFORSENATE/status/1498301886813900804      1
https://twitter.com/Stalingrad_Poor/status/1485656322305445890     1
https://twitter.com/DividendGrowth/status/1460410996485509123      1
Name: quote_url, Length: 768, dtype: int64

In [15]:
#Counted quantity of links for each tweet
BH_tweets_during_war_clean['quote_url'] = BH_tweets_during_war_clean['quote_url'].str.count('https://')

#Replayce NaN by 0
BH_tweets_during_war_clean.fillna({'quote_url':0}, inplace=True)

#Changed data type from float64 to int64
BH_tweets_during_war_clean['quote_url'] = BH_tweets_during_war_clean['quote_url'].astype(np.int64)

In [16]:
#Replayce NaN to [] in thumbnail column
BH_tweets_during_war_clean.fillna({'thumbnail':'[]'}, inplace=True)

In [17]:
#There is no NaN except column place which will be removed due to low importance
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35478 entries, 0 to 35477
Data columns (total 26 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               35478 non-null  int64         
 1   conversation_id  35478 non-null  int64         
 2   created_at       35478 non-null  object        
 3   date             35478 non-null  datetime64[ns]
 4   time             35478 non-null  object        
 5   timezone         35478 non-null  int64         
 6   user_id          35478 non-null  int64         
 7   username         35478 non-null  object        
 8   name             35478 non-null  object        
 9   place            572 non-null    object        
 10  tweet            35478 non-null  object        
 11  language         35478 non-null  object        
 12  mentions         35478 non-null  object        
 13  urls             35478 non-null  object        
 14  photos           35478 non-null  objec

In [18]:
#Check duplicates
BH_tweets_during_war_clean[BH_tweets_during_war_clean.duplicated(subset = 'id')]

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,reply_to


In [19]:
#Column retweet consist of only False values
BH_tweets_during_war_clean['retweet'].value_counts()

False    35478
Name: retweet, dtype: int64

In [20]:
#Removed irrelevant columns
BH_tweets_during_war_clean = BH_tweets_during_war_clean.drop(['conversation_id', 
                                                                        'created_at', 'time', 'timezone', 
                                                                        'language', 'retweet', 'link','place'], 
                                                                       axis = 1)

In [21]:
#Counted number of elements stored as lists in columns
columns_lists = ['mentions', 'urls', 'photos', 'hashtags', 'cashtags', 'thumbnail']
for column in columns_lists:
    empty_list = BH_tweets_during_war_clean[column] == '[]'
    BH_tweets_during_war_clean[column] = BH_tweets_during_war_clean[column].str[1:-1].str.split(',')
    BH_tweets_during_war_clean[column] = BH_tweets_during_war_clean[column].str.len()
    BH_tweets_during_war_clean.loc[empty_list, column] = 0

In [22]:
#Counted number of elements in dictionaries stored in reply_to column
BH_tweets_during_war_clean['reply_to'] = BH_tweets_during_war_clean['reply_to'].str.count('{')

#Changed data type from float64 to int64
BH_tweets_during_war_clean['reply_to'] = BH_tweets_during_war_clean['reply_to'].astype(np.int64)

In [23]:
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35478 entries, 0 to 35477
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              35478 non-null  int64         
 1   date            35478 non-null  datetime64[ns]
 2   user_id         35478 non-null  int64         
 3   username        35478 non-null  object        
 4   name            35478 non-null  object        
 5   tweet           35478 non-null  object        
 6   mentions        35478 non-null  int64         
 7   urls            35478 non-null  int64         
 8   photos          35478 non-null  int64         
 9   replies_count   35478 non-null  int64         
 10  retweets_count  35478 non-null  int64         
 11  likes_count     35478 non-null  int64         
 12  hashtags        35478 non-null  int64         
 13  cashtags        35478 non-null  int64         
 14  quote_url       35478 non-null  int64         
 15  vi

In [24]:
#Analysis of opinion and number of words using the textblob package
for i in range(BH_tweets_during_war_clean.shape[0]):
    blob = TextBlob(BH_tweets_during_war_clean.loc[i, 'tweet'], analyzer=PatternAnalyzer())
    BH_tweets_during_war_clean.loc[i, 'tweet_word_count'] = len(blob.words)
    BH_tweets_during_war_clean.loc[i, 'polarity'] = blob.sentiment[0]
    BH_tweets_during_war_clean.loc[i, 'subjectivity'] = blob.sentiment[1]

In [25]:
BH_tweets_during_war_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35478 entries, 0 to 35477
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                35478 non-null  int64         
 1   date              35478 non-null  datetime64[ns]
 2   user_id           35478 non-null  int64         
 3   username          35478 non-null  object        
 4   name              35478 non-null  object        
 5   tweet             35478 non-null  object        
 6   mentions          35478 non-null  int64         
 7   urls              35478 non-null  int64         
 8   photos            35478 non-null  int64         
 9   replies_count     35478 non-null  int64         
 10  retweets_count    35478 non-null  int64         
 11  likes_count       35478 non-null  int64         
 12  hashtags          35478 non-null  int64         
 13  cashtags          35478 non-null  int64         
 14  quote_url         3547

In [26]:
#Save cleaned data in csv format
BH_tweets_during_war_clean.to_csv('dane/BH_tweets_during_war_clean.csv', index=False)